In this notebook, we will learn to implement a simple differentially private matrix factorization from scratch, with three different strategies:
* input perturbation with Laplacian mechanism
* gradient perturbation with Laplacian mechanism

#### Import packages

In [1]:
!pip install pandas tqdm matplotlib

import sys
sys.path.append('..')

import requests
import os
import io
import zipfile
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from operator import itemgetter
from utils import create_maps, splitting

np.random.seed(42)

  Using cached matplotlib-3.3.4-cp36-cp36m-manylinux1_x86_64.whl (11.5 MB)
     |████████████████████████████████| 3.0 MB 8.4 MB/s eta 0:00:01
  Using cached kiwisolver-1.3.1-cp36-cp36m-manylinux1_x86_64.whl (1.1 MB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)


#### Load Data

First, we download the latest version of the Movielens Small dataset

In [2]:
url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
print(f"Getting Movielens Small from : {url} ...")
response = requests.get(url)

ml_ratings = []

print("Extracting ratings...")
with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
    for line in zip_ref.open("ml-latest-small/ratings.csv"):
        ml_ratings.append(str(line, "utf-8"))

print("Printing ratings to data/movielens/ ...")
os.makedirs("data/movielens", exist_ok=True)
with open("data/movielens/dataset.csv", "w") as f:
    f.writelines(ml_ratings)

Getting Movielens Small from : https://files.grouplens.org/datasets/movielens/ml-latest-small.zip ...
Extracting ratings...
Printing ratings to data/movielens/ ...


Processing dataset

In [3]:
dataframe_ml_small = pd.read_csv('data/movielens/dataset.csv')

train_set, test_set = splitting(dataframe_ml_small)
train_set = train_set.loc[:, ['userId', 'movieId', 'rating']]
test_set = test_set.loc[:, ['userId', 'movieId', 'rating']]
maps = create_maps(train_set)

Performing splitting...
Done!


### Define the model

Create MF class

In [4]:
class MF:
    def __init__(self, dataset, maps, n_factors, relevance=3.5, i_avg=None, u_avg=None):
        """
        :param dataset: interaction dataset should be a Pandas dataframe with three columns for user, item, and rating
        :param n_factors:
        """
        print("Building model...")
        self.ext2int_user_map, self.int2ext_user_map, self.ext2int_item_map, self.int2ext_item_map = maps
        self.dataset = self.format_dataset(dataset)
        self.rated_items = {
            self.ext2int_user_map[u]: dataset[(dataset.iloc[:, 0] == u) & (dataset.iloc[:, 2] >= relevance)].iloc[:,
                                      1].map(self.ext2int_item_map).astype(int).to_list() for u in
            self.ext2int_user_map}
        n_users = len(self.ext2int_user_map)
        n_items = len(self.ext2int_item_map)
        self.n_interactions = len(dataset)
        self.delta_ratings = dataset.iloc[:, 2].max() - dataset.iloc[:, 2].min()
        self.p = np.random.normal(size=(n_users, n_factors), scale=1./n_factors, loc=0)
        self.q = np.random.normal(size=(n_items, n_factors), scale=1./n_factors, loc=0)

        self.b_u = np.zeros(n_users)
        self.b_i = np.zeros(n_items)
        self.b = np.mean(dataset['rating'])

        self.i_avg = None
        self.u_avg = None
        if i_avg is not None and u_avg is not None:
            self.i_avg = i_avg.to_numpy()
            self.u_avg = u_avg.to_numpy()

    def format_dataset(self, df):
        dataset = {}
        dataset['userId'] = df.iloc[:, 0].map(self.ext2int_user_map).to_dict()
        dataset['itemId'] = df.iloc[:, 1].map(self.ext2int_item_map).to_dict()
        dataset['rating'] = df.iloc[:, 2].to_dict()
        return dataset

    def train(self, lr, beta, epochs):
        print("Starting training...")
        for e in range(epochs):
            print(f"*** Epoch {e + 1}/{epochs} ***")
            for i in tqdm(range(self.n_interactions)):
                p_u = self.p[self.dataset['userId'][i]]
                q_i = self.q[self.dataset['itemId'][i]]
                pred = self.b + self.b_u[self.dataset['userId'][i]] + self.b_i[self.dataset['itemId'][i]] + p_u.dot(q_i)
                err = self.dataset['rating'][i] - pred

                # Update biases
                self.b_u[self.dataset['userId'][i]] += lr * (err - beta * self.b_u[self.dataset['userId'][i]])
                self.b_i[self.dataset['itemId'][i]] += lr * (err - beta * self.b_i[self.dataset['itemId'][i]])

                # Update embeddings
                self.p[self.dataset['userId'][i]] = p_u + lr * (err * q_i - beta * p_u)
                self.q[self.dataset['itemId'][i]] = q_i + lr * (err * p_u - beta * q_i)

    def train_laplace_dp(self, lr, beta, epochs, eps, err_max=None):
        for e in range(epochs):
            print(f"*** Epoch {e + 1}/{epochs} ***")
            for i in tqdm(range(self.n_interactions)):
                p_u = self.p[self.dataset['userId'][i]]
                q_i = self.q[self.dataset['itemId'][i]]
                pred = self.b + self.b_u[self.dataset['userId'][i]] + self.b_i[self.dataset['itemId'][i]] + p_u.dot(q_i)

                # We should add Laplacian noise scaled based on the sensitivity of the error
                # Having two datasets A and B differing by the value of just one rating r_ui (namely r_ui(A) and r_ui(B)), we have that
                # max |e(A) - e(B)| = max |(r_ui(A) - p_u * q_i) - (r_ui(B) - p_u * q_i)| = max |r_ui(A) - r_ui(B)| = delta_ratings
                # For the composability theorem, we have to split the privacy budget by the number of epochs
                err = self.dataset['rating'][i] - pred + np.random.laplace(scale=(epochs * self.delta_ratings / eps))

                # Optionally we can constrain the error the limit the effect of the noise
                if err_max:
                    err = np.clip(err, -err_max, err_max)

                # Update biases
                self.b_u[self.dataset['userId'][i]] += lr * (err - beta * self.b_u[self.dataset['userId'][i]])
                self.b_i[self.dataset['itemId'][i]] += lr * (err - beta * self.b_i[self.dataset['itemId'][i]])

                self.p[self.dataset['userId'][i]] = p_u + lr * (err * q_i)
                self.q[self.dataset['itemId'][i]] = q_i + lr * (err * p_u)


    def evaluate(self, test=None, cutoff=10, relevance=0.5):
        print("Starting evaluation...")
        if self.i_avg is not None and self.u_avg is not None:
            prediction = self.b + self.b_u[:, np.newaxis] + self.b_i[np.newaxis, :] + \
                         (np.dot(self.p, self.q.T).T + self.i_avg[:, None]).T + self.u_avg[:, None]
        else:
            prediction = self.b + self.b_u[:, np.newaxis] + self.b_i[np.newaxis, :] + np.dot(self.p, self.q.T)
        precisions = []
        recalls = []
        print("Reading test set...")
        relevant_items_test = {
            self.ext2int_user_map[u]: set(test[(test.iloc[:, 0] == u) & (test.iloc[:, 2] >= relevance)].iloc[:, 1].map(
                self.ext2int_item_map).dropna().astype(int).to_list()) for u in self.ext2int_user_map}
        print("Computing metrics...")
        for u in self.int2ext_user_map:
            prediction[u, self.rated_items[u]] = - np.inf
            unordered_top_k = np.argpartition(prediction[u], -cutoff)[-cutoff:]
            top_k = unordered_top_k[np.argsort(prediction[u][unordered_top_k])][::-1]
            n_rel_and_rec_k = sum(i in relevant_items_test[u] for i in top_k)
            precisions.append(n_rel_and_rec_k / cutoff)
            try:
                recalls.append(n_rel_and_rec_k / len(relevant_items_test[u]))
            except ZeroDivisionError:
                recalls.append(0)
        precision = sum(precisions) / len(precisions)
        recall = sum(recalls) / len(recalls)

        print(f"Precision@{cutoff}: {precision}")
        print(f"Recall@{cutoff}: {recall}")
        return  precision, recall

#### Initialize and Train The Model
Now, we are ready to initialize and train the model.

In [5]:
f = 100
lr = 0.001
lmb = 0.1
epochs = 20

mf = MF(train_set, maps, f, relevance=4)
mf.train(lr, lmb, epochs)

Building model...
Starting training...
*** Epoch 1/20 ***


100%|██████████| 80419/80419 [00:01<00:00, 61992.52it/s]


*** Epoch 2/20 ***


100%|██████████| 80419/80419 [00:01<00:00, 59647.78it/s]


*** Epoch 3/20 ***


...

100%|██████████| 80419/80419 [00:01<00:00, 59257.15it/s]


*** Epoch 19/20 ***


100%|██████████| 80419/80419 [00:01<00:00, 60611.95it/s]


*** Epoch 20/20 ***


100%|██████████| 80419/80419 [00:01<00:00, 57361.42it/s]


#### Evaluate The Model

The evaluation is computed on Top-K recommendation lists (default K = 10).

In [6]:
mf.evaluate(test_set)

Starting evaluation...
Reading test set...
Computing metrics...
Precision@10: 0.058524590163934534
Recall@10: 0.03103142768420843


(0.058524590163934534, 0.03103142768420843)

Build DP schema on dataset

Before feeding our recommender, we preprocess the dataset by measuring noisy versions of some global effects. In detail we measure:
* a differentially private version of the global average
* a differentially private version of the item averages
* a differentially private version of the user averages

Finally, we clamp the resulting ratings

This preprocessing is proven to allow deriving more accurate predictions when using the MF approach

In [8]:
def privatize_global_effects(ratings, b_m, b_u, eps_global_avg, eps_item_avg, eps_user_avg, clamping):
    min_rating = ratings['rating'].min()
    max_rating = ratings['rating'].max()
    delta_r = max_rating - min_rating

    # Measure the noisy version

    global_average_item = (ratings['rating'].sum() + np.random.laplace(scale=(delta_r / eps_global_avg))) / len(ratings)

    item_sets = ratings.groupby('movieId')['rating']
    i_avg = (item_sets.sum() + b_m * global_average_item + np.random.laplace(scale=(delta_r / eps_item_avg),
                                                                             size=len(item_sets))) / (
                        item_sets.count() + b_m)
    i_avg = np.clip(i_avg, min_rating, max_rating)

    merged = ratings.join(i_avg, on=['movieId'], lsuffix='_x', rsuffix='_y')

    merged['rating'] = merged['rating_x'] - merged['rating_y']
    merged = merged.drop(columns=['rating_x', 'rating_y'], axis=1)

    global_average_user = (merged['rating'].sum() + np.random.laplace(scale=(delta_r / eps_global_avg))) / len(merged)

    user_sets = merged.groupby('userId')['rating']
    u_avg = (user_sets.sum() + b_u * global_average_user + np.random.laplace(scale=(delta_r / eps_user_avg))) / (
                user_sets.count() + b_u)
    u_avg = np.clip(u_avg, -2, 2)
    # Values come from the implementation of this approach by Friedman et al., 2016

    preprocessed_ratings = merged.join(u_avg, on=['userId'], lsuffix='_x', rsuffix='_y')

    preprocessed_ratings['rating'] = preprocessed_ratings['rating_x'] - preprocessed_ratings['rating_y']
    preprocessed_ratings = preprocessed_ratings.drop(columns=['rating_x', 'rating_y'], axis=1)
    preprocessed_ratings['rating'] = np.clip(preprocessed_ratings['rating'], -clamping, clamping)

    return preprocessed_ratings, i_avg, u_avg

From Friedman et. al, 2016, having a eps privacy budget, we can split it with the following proportion:

In [9]:
eps = 5
eps_global_avg = 0.02 * eps
eps_item_avg = 0.14 * eps
eps_user_avg = 0.14 * eps
# Overall, we used 0.3 of our eps for the preprocessing
# The remaing 0.7 of eps is used for training

Let's preprocess the data based on the protocol we analyzed so far

In [10]:
b_m = 5
b_u = 5
clamping = 1
preproc_train_set, i_avg, u_avg = privatize_global_effects(train_set, b_m, b_u, eps_global_avg, eps_item_avg,
                                                           eps_user_avg, clamping)

Let's explore the first perturbation strategy, applied on the inputs

In [11]:
def input_perturbation(ratings, clamping, eps):
    # Range of the received ratings is [-clamping, clamping]
    # In the input perturbation approach, each rating is perturbed in a way that maintains differential privacy
    delta_r = 2 * clamping  # global sensitivity

    # differential privacy
    perturbed_ratings = ratings.copy()
    perturbed_ratings['rating'] = np.clip(ratings['rating'] + np.random.laplace(scale=(delta_r / eps), size=len(ratings)),
                                -clamping, clamping)

    return perturbed_ratings

train_set_perturbed = input_perturbation(train_set, 1, 0.7 * eps)
mf_dp_data = MF(train_set_perturbed, maps, f, relevance=4, i_avg=i_avg, u_avg=u_avg)
mf_dp_data.train(lr, lmb, epochs)
mf_dp_data.evaluate(test_set)

Building model...
Starting training...
*** Epoch 1/20 ***


100%|██████████| 80419/80419 [00:01<00:00, 45030.22it/s]


*** Epoch 2/20 ***


100%|██████████| 80419/80419 [00:01<00:00, 51567.86it/s]


*** Epoch 3/20 ***


...

100%|██████████| 80419/80419 [00:01<00:00, 52393.54it/s]


*** Epoch 20/20 ***


100%|██████████| 80419/80419 [00:01<00:00, 57896.69it/s]


Starting evaluation...
Reading test set...
Computing metrics...
Precision@10: 0.020163934426229473
Recall@10: 0.009735975249187529


(0.020163934426229473, 0.009735975249187529)

Let's explore the second approach, with the differential privacy applied to the SGD algorithm

In [12]:
mf_dp_sgd = MF(train_set, maps, f, relevance=4, i_avg=i_avg, u_avg=u_avg)
mf_dp_sgd.train_laplace_dp(lr, lmb, epochs, 0.7 * eps)
mf_dp_sgd.evaluate(test_set)


Building model...
*** Epoch 1/20 ***


100%|██████████| 80419/80419 [00:01<00:00, 46285.38it/s]


*** Epoch 2/20 ***


100%|██████████| 80419/80419 [00:01<00:00, 47689.42it/s]


*** Epoch 3/20 ***


...

100%|██████████| 80419/80419 [00:01<00:00, 44706.27it/s]


*** Epoch 9/20 ***


  0%|          | 137/80419 [00:00<00:03, 26357.49it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/claudio/PycharmProjects/RecSysPrivacyTutorial/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-1cf3ab10aa07>", line 2, in <module>
    mf_dp_sgd.train_laplace_dp(lr, lmb, epochs, 0.7 * eps)
  File "<ipython-input-4-4d0bace1df81>", line 75, in train_laplace_dp
    self.b_u[self.dataset['userId'][i]] += lr * (err - beta * self.b_u[self.dataset['userId'][i]])
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/claudio/PycharmProjects/RecSysPrivacyTutorial/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Tra

TypeError: object of type 'NoneType' has no len()

In [ ]:
eps_valuse = [0.2, 1, 5, 10]
results = {}
for eps in eps_valuse:
    print(f"*** start eps {eps} ***")
    eps_global_avg = 0.02 * eps
    eps_item_avg = 0.14 * eps
    eps_user_avg = 0.14 * eps
    b_m = 5
    b_u = 5
    clamping = 1
    preproc_train_set, i_avg, u_avg = privatize_global_effects(train_set, b_m, b_u, eps_global_avg, eps_item_avg,
                                                               eps_user_avg, clamping)
    train_set_perturbed = input_perturbation(train_set, 1, 0.7 * eps)
    mf_dp_data = MF(train_set_perturbed, maps, f, relevance=4, i_avg=i_avg, u_avg=u_avg)
    mf_dp_data.train(lr, lmb, epochs)
    p_d, r_d = mf_dp_data.evaluate(test_set)

    mf_dp_sgd = MF(train_set, maps, f, relevance=4, i_avg=i_avg, u_avg=u_avg)
    mf_dp_sgd.train_laplace_dp(lr, lmb, epochs, 0.7 * eps)
    p_m, r_m = mf_dp_sgd.evaluate(test_set)

    results.update({eps: (p_d, r_d, p_m, r_m)})
    print(f"*** end eps {eps} ***")

In [ ]:
for i in range(len(results[0.2])):
    plt.plot(results.keys(), [v[i] for k, v in results.items()])

plt.title('Precision across eps')
plt.xlabel('eps')